# Lecture 17: Partial Differential Equations - Elliptic 🌡️

## 1. Steady-State 2D Temperature of a Plate

$$\frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2} = 0$$

Constant (Dirichlet) BCs are: $B_l=75$, $B_u=100$, $B_r=50$, $B_d=0$. Units are Celsius.

<p align="center">
  <img src="https://github.com/cdefinnda/ECI-115_HW-Images/blob/main/NB_L17.png?raw=true" alt="NB_L17" width=1000>
</p>

In [ ]:
%reset -f
# Import Libraries
import numpy as np
import matplotlib.pyplot as plt

💪 Discretize the Domain to Create Matrix $\mathbf{A}$

In [ ]:
# Grid Size (n x n)
n = 3 # Define this so that we can change the resolution of our 2D solution.

# LHS Matrix A (bandwidth = 2n+1). Make each diagonal separately:
main = #[insert code here]
off1 = #[insert code here]
off1[n-1:-1:n] = #[insert code here]
offn = #[insert code here]
A = #[insert code here]

if n <= 4:
    print(A) # confirm same as lecture notes, do not print for larger n

💪 Use Boundary Conditions to Create the Vector $\mathbf{b}$

There are two ways to do this (we'll do Option 2):
1. Create a $n^2 \times 1$ vector and assign the index values from the lecture notes
2. Create a $n \times n$ matrix, assign the boundaries, and then reshape to $n^2 \times 1$.

In [ ]:
# Create RHS vector b with boundary conditions
Bd,Bl,Br,Bu = (0,75,50,100)

# Create BC Matrix
# Note that even though this matrix would appear upside-down, we want to make sure the indicies are correct.
b = np.zeros((n,n))
b[:,0] += #[insert code here]
b[:,-1] += #[insert code here]
b[0,:] += #[insert code here]
b[-1,:] += #[insert code here]

b = np.reshape(b, (n**2,1)) # This flattens our matrix into an array based on row index then column index

if n <= 4:
    print(b)

🤝 Solve for $\mathbf{x}$
* Use the built-in function: `np.linalg.solve(A,b)`.
* Note $\mathbf{A}$ is sparse, so for large values of $n$ it may be more efficient to use an iterative method such as Gauss-Seidel.

In [ ]:
T = np.linalg.solve(A,b) # returns n^2 x 1 column vector
T = np.reshape(T,(n,n)) # Fills in an n x n matrix in order based on row index then column index.

In [ ]:
plt.imshow(T, origin='lower', cmap = 'magma') # default puts origin at top-left
plt.colorbar(label='Temperature (°C)')
plt.title('Steady-State Temperature of a Heated Plate')
plt.xlabel('X index')
plt.ylabel('Y index')
plt.show()

---
## 2. Add a Point Source

🤝 Add point source $T=75$ in the center of the grid.
* We only need to modify $\mathbf{b}$, not $\mathbf{A}$.
* The index in our $\mathbf{A}$ matrix ($i_A$, $j_A$) maps to the index of our vector $\mathbf{b}$ ($k_b$) by the following relationship: $k_b = (n\cdot i_A) + j_A$

In [ ]:
# Create a copy of b that we can modify
b2 = b.copy()
T_point = 75

# Find Center Index of b2
i_A = #[insert code here]
j_A = #[insert code here]
index = #[insert code here]

b2[index] += T_point

In [ ]:
# Note that the previous code only works if n is an odd number: n % 2 == 1
# If n is even, we may want to set our central 4 nodes as the point source
if n % 2 == 0:
    b2[index] -= T_point # Would need to run this after the previous code block
    i_center = n // 2
    j_center = n // 2
    # 4 center points (top-left, top-right, bottom-left, bottom-right of center block)
    center_indices = [
        (i_center - 1, j_center - 1),
        (i_center - 1, j_center),
        (i_center,     j_center - 1),
        (i_center,     j_center)
    ]
    for i, j in center_indices:
        index = i * n + j
        b2[index] += T_point /4

In [ ]:
# Solve
T = np.linalg.solve(A,b2)
T = np.reshape(T,(n,n))

# Plot Results
plt.imshow(T, origin='lower', cmap = 'magma')
plt.colorbar(label='Temperature (°C)')
plt.title('Steady-State Temperature of a Heated Plate')
plt.xlabel('X index')
plt.ylabel('Y index')
plt.show()

---
## 3. Apply Neumann BC

🤝 Repeat with the lower boundary condition changed to no-flux, $\frac{\partial T}{\partial y}=0$.

Our finite difference equation at the lower boundary becomes:
* Dirichlet BC: $4T_{i,0}-T_{i-1,0}-T_{i,-1}-T_{i,1}=0$
* Neumann BC: $4T_{i,0}-T_{i-1,0}-2T_{i,1}=0$

This gives us $n$ more unknowns, so our matrix size is now $n^2 + n$.

In [ ]:
# Clear Variables
%reset -f

# Import Libraries
import numpy as np
import matplotlib.pyplot as plt


n = 3 # grid size (n^2 internal nodes)
d = n**2 + n # dimension of unknowns

# LHS matrix A
main = 4*np.ones(d)
off1 = -1*np.ones(d-1)
off1[n-1:-1:n] = 0 # account for zeros on +/-1 diagonal
offnup = -1*np.ones(d-n) # the +/- nth diagonal contains d-n elements
offndown = -1*np.ones(d-n)

# First n rows: the +nth diagonal is -2 to enforce the lower BC
offnup[:n] = -2

A = np.diag(main) + np.diag(off1,1) + np.diag(off1,-1) + np.diag(offnup,n) + np.diag(offndown,-n)

# RHS BCs (note no Bd anymore)
Bl,Br,Bu = (75,50,100)
b = np.zeros((n+1,n)) # now n+1 rows
b[:,0] += Bl
b[:,-1] += Br
b[-1,:] += Bu
b = np.reshape(b, (d,1))

#print(A)

In [ ]:
# Solve
T = np.linalg.solve(A,b)
T = np.reshape(T,(n+1,n))

# Plot Results
plt.imshow(T, origin='lower', cmap = 'magma')
plt.colorbar(label='Temperature (°C)')
plt.title('Temperature w/no-flux lower BC')
plt.xlabel('X index')
plt.ylabel('Y index')
plt.show()

**Note**: In all of the above cases, our plots only show the internal nodes where we are solving for $T$. We could also append the BCs around the edge of the grid.

---
## Bonus: Alternative Solving & Plotting Methods

### Iterative Solution
As we mentioned before, we could also use  Gauss-Seidel iterations (Liebmann's method) to solve.

This method is good for large $n$ and irregular geometries, such as HW 8 Problem 32.4.

In [ ]:
# start from scratch with Dirichlet BC problem
import scipy.linalg as sl

Bd,Bl,Br,Bu = (0,75,50,100)
n = 50
err = 9999
it = 0 # iteration count
T = np.zeros((n,n))

while err > 1e-1:
  T_old = T.copy() # store to calculate error
  for i in range(1,n-1): # exclude BCs
    for j in range(1,n-1):
      T[i,j] = (T[i+1,j] + T[i-1,j] + T[i,j+1] + T[i,j-1]) / 4

  T[:,0] = Bl # BCs around edge
  T[:,-1] = Br
  T[0,:] = Bd
  T[-1,:] = Bu
  # for Neumann no-flux BCs, set T[0,:] = T[1,:] instead

  err = sl.norm(T - T_old, 2)
  it += 1

print(it, ' iterations')
plt.imshow(T, origin='lower')
plt.colorbar(label='Temperature (°C)')
plt.show()

---
## Other Types of Plots

### Contour Plot

In [ ]:
plt.contour(T, levels=20)
plt.title('Temperature Contour Plot')
plt.xlabel('X index')
plt.ylabel('Y index')
plt.colorbar(label='Temperature (°C)')
plt.show()

### Surface Plot

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})

X = np.arange(0,n)
Y = np.arange(0,n)
X, Y = np.meshgrid(X, Y) # creates a 2D grid from 1D vectors
ax.plot_surface(X, Y, T, cmap='viridis')
plt.title('Surface plot of temperature')
plt.xlabel('X index')
plt.ylabel('Y index')
plt.show()